In [1]:
!pip install unidecode
!pip install regex
!pip install stop-words


In [2]:
import pandas as pd
import re
import regex
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from stop_words import get_stop_words
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout, concatenate

ModuleNotFoundError: No module named 'pandas'

In [4]:
# Cargar datos de tweets
tweets_df = pd.read_excel('Tweets.xlsx')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1993140232.py, line 2)

In [17]:
# Definir una función de preprocesamiento
def preprocess_tweet(tweet):
    tweet = re.sub(r'@[A-Za-z0-9_]+', '', tweet)
    tweet = re.sub(r'http\S+', '', tweet)
    tweet = re.sub(r'[^A-Za-z\d\s]+', ' ', tweet)  # Incluir letras, números y espacios
    tweet = tweet.lower()
    return tweet

tweets_df['Tweet'] = tweets_df['Tweet'].astype(str)

# Aplicar preprocesamiento y tokenización
tweets_df['ProcessedTweet'] = tweets_df['Tweet'].apply(preprocess_tweet)


In [18]:
tweets_df

,Tweet,estado,ProcessedTweet
0,Cordobés porque me la complicaste con el cosen...,0.0,cordob s porque me la complicaste con el coseno
1,Tengo fiebre :(,0.0,tengo fiebre
2,@sooooyderiver dame bola :(,0.0,dame bola
3,Quiero bailar salsa :(,0.0,quiero bailar salsa
4,Metal Gear. :(,0.0,metal gear
...,...,...,...
176001,Acá cocinandole a mi Hno :),1.0,ac cocinandole a mi hno
176002,"@nyazfthes Gracias por seguirme, en breve te d...",1.0,gracias por seguirme en breve te devuelvo fo...
176003,Y ahora no podré dormir :))))))),1.0,y ahora no podr dormir
176004,En la Boca :) @ Estadio Boca Juniors https://t...,1.0,en la boca estadio boca juniors


In [19]:
# Eliminar filas con valores NaN en 'estado'
tweets_df = tweets_df.dropna(subset=['estado'])

# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(tweets_df['ProcessedTweet'], tweets_df['estado'], test_size=0.2, random_state=42)

# Descargar las stop words en español
stop_words_es = get_stop_words('spanish')

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=5000, stop_words=stop_words_es)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)




In [20]:
# Entrenar un modelo de Random Forest
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train_tfidf, y_train)
rf_predictions = rf_classifier.predict(X_test_tfidf)

# Evaluar el modelo
accuracy = accuracy_score(y_test, rf_predictions)
print(f'Accuracy del modelo: {accuracy:.2f}')

# Generar un informe de clasificación
print('\nInforme de Clasificación:')
print(classification_report(y_test, rf_predictions))

Accuracy del modelo: 0.79

Informe de Clasificación:
              precision    recall  f1-score   support

         0.0       0.80      0.92      0.85     24033
         1.0       0.74      0.50      0.60     11168

    accuracy                           0.79     35201
   macro avg       0.77      0.71      0.73     35201
weighted avg       0.78      0.79      0.77     35201



In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_test, rf_predictions)

# Mostrar
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
            xticklabels=rf_classifier.classes_,
            yticklabels=rf_classifier.classes_)
plt.title('Matriz de Confusión')
plt.xlabel('Predicciones')
plt.ylabel('Valores Verdaderos')
plt.show()

In [21]:

# Tokenizar y convertir texto a secuencias numéricas para la CNN
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad secuencias para que tengan la misma longitud
max_len = max(len(seq) for seq in X_train_seq + X_test_seq)
X_train_seq = pad_sequences(X_train_seq, maxlen=max_len)
X_test_seq = pad_sequences(X_test_seq, maxlen=max_len)


In [22]:
# Construir el modelo CNN
cnn_model = Sequential()
cnn_model.add(Embedding(input_dim=5000, output_dim=128, input_length=max_len))
cnn_model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
cnn_model.add(GlobalMaxPooling1D())
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(1, activation='sigmoid'))


In [23]:
# Compilar el modelo
cnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo CNN
cnn_model.fit(X_train_seq, y_train, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5
3521/3521 [==============================] - 2784s 790ms/step - loss: 0.4403 - accuracy: 0.8012 - val_loss: 0.4089 - val_accuracy: 0.8140
Epoch 2/5
3521/3521 [==============================] - 2832s 804ms/step - loss: 0.3789 - accuracy: 0.8343 - val_loss: 0.4106 - val_accuracy: 0.8145
Epoch 3/5
 101/3521 [..............................] - ETA: 42:16 - loss: 0.3243 - accuracy: 0.8636

KeyboardInterrupt: ignored